In [1]:
import joblib
import gradio as gr
from PIL import Image
import numpy as np
import io
import cv2
import pandas as pd
import pydicom
from scipy.stats import mode

In [2]:
# model elmentése
# ki kellene találni, hogy mi lesz ha több modelt kell elmentenünk
# szükséges ahhoz, hogy a gradio működjön



In [3]:
def normalize(item):
    pixel = item
    minimum = pixel.min()
    maximum = pixel.max()

    if minimum < 0:
        pixel = pixel + abs(minimum)
        maximum += abs(minimum)
        
    if maximum != 0:
        pixel = pixel / maximum
    return pixel

In [4]:
def preprocess_image(image, dicom, file_format):
    num=1
    if (file_format=='DICOM'):
        num=len(dicom)
    images=[]
    
    for i in range(num):
        if (file_format=='DICOM'):
            dicom_image=pydicom.dcmread(dicom[i])  
            gray_image=dicom_image.pixel_array
        else:
            img = Image.fromarray(np.uint8(image))   
            gray_image = img.convert('L')
        
        image_array = np.array(gray_image)
        resized_image = cv2.resize(image_array, (256, 256))
        normalized_image=normalize(resized_image)
        images.append(normalized_image)

    return images

In [5]:
def preprocessing (gender, age_group, file_format, dicom, image):
    cols=['Is_Male', 'Age_Group_3', 'Age_Group_4', 'Age_Group_5', 'Age_Group_6', 'Age_Group_7', 'Age_Group_8', 'Image']
    cols_age=[ 'Age_Group_3', 'Age_Group_4', 'Age_Group_5', 'Age_Group_6', 'Age_Group_7', 'Age_Group_8']
    df = pd.DataFrame(columns=cols)
    
    #image
    proc_img=preprocess_image(image, dicom, file_format)
    if (len(proc_img)==1):
        df.loc[0, 'Image']=proc_img
    else:
        for i in range(len(proc_img)):
            df.loc[i, 'Image']=proc_img[i]

    df['Image'] = df['Image'].apply(lambda x: np.array(x).flatten())
    
    #age_group
    for item in cols_age:
        if (int(item[10])==int(age_group[0])-1):
            df[item]=True
        else:
            df[item]=False
    #gender
    if (gender=='Male'):
        df['Is_Male']=True
    else:
        df['Is_Male']=False

    return df

In [6]:
def model_predict(gender, age_group, file_format, dicom, image):
    df=preprocessing(gender, age_group, file_format, dicom, image)
    image_classifier=joblib.load('model_image.joblib')
    patient_data_classifier=joblib.load('model_patient_data.joblib')

    df_image=df['Image'].tolist()
    df_patient_data=df.drop('Image', axis=1)

    y_image=image_classifier.predict(df_image)

    labels = {0: 'Normal', 1: 'Heart failure with infarct', 2: 'Heart failure without infarct', 3: 'Hypertrophy'}

    if (len(y_image)==1):
        return labels[y_image[0]]
    else:
        result=mode(y_image)
        string=f"{labels[result.mode]} with an {result.count/len(y_image)*100}% certainty"
        
        return string

In [7]:
options_gender= ['Female', 'Male']
options_age_group=['40 or younger', '41-50', '51-60', '61-70', '71-80', '81 or older']
options_file_upload=['DICOM', 'Other Image Format']

demo = gr.Interface(
    fn=model_predict,
    inputs=[
        gr.Radio(options_gender, label="Patient's gender"),
        gr.Radio(options_age_group, label="Patient's age"),
        gr.Radio(options_file_upload, label="Choose your file format! You can upload several files with dicom format, but only one image"), 
        gr.File(label="Upload your DICOM file!", file_count='multiple'),
        gr.Image(label='Upload your picture!')],
    outputs=gr.Textbox(label="Diagnosis")
)
    
if __name__ == "__main__":
    demo.launch(inbrowser=True, show_api=False)   

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
